In [1]:
from IPython.display import display, Math, Latex
# This is imported for proper rendering of Latex in Notebook.

import numpy as np

# import for generating plots
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Multi-Output/Multi-Label Regression

In case of multi-output regression, there are more than one output labels, all of which are real numbers.

## Training Data

$D = (X,Y) = \{(x^{(i)},y^{(i)})\}^{n}_{i=0}$

Here $y^{(i)} \in \mathbb{R}^k$ where $k$ is the no. of output labels.

### Generating Synthetic Data

Let's generate synthetic data for demonstrating the training set in multi-output regression using `sklearn.datasets.make_regression` function.

In [7]:
from sklearn.datasets import make_regression

X,Y,coef = make_regression(n_samples=100, n_features=10, n_informative=10, bias=1, n_targets=5, shuffle=True, coef=True, random_state=42)

Let's check the shape of input and output.

In [8]:
print(X.shape, Y.shape)

(100, 10) (100, 5)


Let's examine the first $5$ examples in terms of their features and labels:

In [9]:
print("Sample training examples:\n", X[:5])
print("Corresponding labels:\n", Y[:5])

Sample training examples:
 [[-2.07339023 -0.37144087  1.27155509  1.75227044  0.93567839 -1.40751169
  -0.77781669 -0.34268759 -1.11057585  1.24608519]
 [-0.90938745 -1.40185106 -0.50347565 -0.56629773  0.09965137  0.58685709
   2.19045563  1.40279431 -0.99053633  0.79103195]
 [-0.18565898 -1.19620662 -0.64511975  1.0035329   0.36163603  0.81252582
   1.35624003 -1.10633497 -0.07201012 -0.47917424]
 [ 0.03526355  0.21397991 -0.57581824  0.75750771 -0.53050115 -0.11232805
  -0.2209696  -0.69972551  0.6141667   1.96472513]
 [-0.51604473 -0.46227529 -0.8946073  -0.47874862  1.25575613 -0.43449623
  -0.30917212  0.09612078  0.22213377  0.93828381]]
Corresponding labels:
 [[-133.15919852  -88.95797818   98.19127175   25.68295511 -132.79294654]
 [-110.38909784  146.04459736 -169.58916067  118.96066861 -177.08414159]
 [ -97.80350267    4.32654061  -87.56082281   -5.58466452    6.36897388]
 [  25.39024616  -70.41180117  186.15213706  132.77153362   53.42301307]
 [-140.61925153  -53.87007831 -1

And the coefficients or weight vectors used for generating this dataset is

In [11]:
print(coef)
print(coef.shape)

[[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 73.15895218  8.1629982   6.0352084  24.7103234 ]
 [15.95446801 87.17835666 21.92139874 97.58652558 33.68957918]
 [71.40869321 80.17280831 33.94501925 81.48251137  8.01148464]
 [18.21179157 78.96985071 65.87077755 49.81957165 55.53635509]
 [16.74825823 10.45678403 63.64302495 70.64757265  3.15861448]]
(10, 5)


Since, we have $5$ labels per output and each each input has $10$ features, we get a `(10,5)` weight vector signifying that we have 10 weight coefficients per output label.

This in some sense tells us that this is going to be equivalent to running linear regression $5$ times, once per output label and combinig the weight vector in the end.

### Data Preprocessing: Dummy feature and train-test split

In [12]:
def add_dummy_feature(X:np.ndarray):
    ''' Adds a dummy feature to the dataset.
    
    Args:
        X: Training dataset
    
    Returns:
        Training dataset with the addition of dummy feature.
    '''
    return np.column_stack((np.ones(X.shape[0]), X))

def preprocess(X:np.ndarray,y:np.ndarray):
    ''' Adds a dummy feature to the dataset. Then splits the dataset into 2 parts --- 80% for training and 20% for testing.
    
    Args:
        X: Training dataset
        y: Label vector
    
    Returns:
        Training dataset with the addition of dummy feature.
        Testing dataset with the addition of dummy feature.
        Training label vector.
        Testing label vector.
    '''
    X_with_dummy_features = add_dummy_feature(X)

    from sklearn.model_selection import train_test_split
    
    return train_test_split(X_with_dummy_features, y, test_size=0.20, random_state=42)

In [13]:
X_train, X_test, Y_train, Y_test = preprocess(X,Y)

In [14]:
print(X_train.shape,X_test.shape)
print(Y_train.shape, Y_test.shape)

(80, 11) (20, 11)
(80, 5) (20, 5)


## Model

The linear regression model is exactly the same as a single label output except that the output now is a vector:
$$Y_{n \times k} = X_{n \times (m+1)} W_{(m+1) \times k}$$

In this equation, the output label is a matrix $Y$. In order to generate multiple outputs, we nee one weight vector per output. Hence, total of $k$ weight vectors corresponding to the $k$ outputs.

There are $2$ options for modelling this problem:
1. Solve $k$ independent linear regression problems. Gives some flexibility in using different representation for each problem.
2. Solve a joint learning problem as outined in the equation above. We will pursue this approach in this notebook.

### Loss

We use the same loss function as the linear regression i.e. sum of squared error.
$$J(W) = \frac{1}{2} (XW - Y)^T (XW - Y)$$

### Optimization

1. Normal Equation
2. Gradient descent and its variations

### Evaluation

RMSE or Squared Loss

## Linear Regression Implementation

In [27]:
class LinReg(object):
    '''
    Modified Linear Regression model
    -----------------------
    y = X@w
    X: Feature matrix
    w: Weight vector
    y: Label vector
    λ: Regularization Rate
    '''

    def __init__(self):
        self.t0 = 200
        self.t1 = 100000
    
    def predict(self, X:np.ndarray) -> np.ndarray:
        '''Prediction of output label for a given input.

        Args:
            X: Feature matrix for given inputs
        
        Returns:
            y: Output label vector as predicted by the given model
        '''

        y = X @ self.w
        
        return y
    
    def loss(self, X:np.ndarray, y:np.ndarray, reg_rate:float=0) -> float:
        '''Calculate loss for a model based on actual labels.

        Args:
            X: Feature matrix for given inputs
            y: Output label vector as known from the dataset (actual labels)
            reg_rate: Rate of regularization
        
        Returns:
            Loss
        '''

        e = y - self.predict(X)
        # return (1/2) * (np.transpose(e) @ e)
        return (1/2) * (np.transpose(e) @ e) + (reg_rate/2) * (np.transpose(self.w) @ self.w)
    
    def rmse(self, X:np.ndarray, y:np.ndarray, reg_rate:float=0) -> float:
        '''Calculate root mean squared error of prediction based on actual labels.

        Args:
            X: Feature matrix for given inputs
            y: Output label vector as known from the dataset (actual label)
            reg_rate: Rate of regularization
        
        Returns:
            RMSE
        '''
        return np.sqrt((2/X.shape[0]) * self.loss(X,y,reg_rate))
    
    # Normal Equation based estimation
    def fit(self, X:np.ndarray, y:np.ndarray, reg_rate:float=0) -> np.ndarray:
        '''Estimate parameters of linear regression model with normal equation.

        Args:
            X: Feature matrix for given inputs
            y: Output label vector as known from the dataset (actual label)
            reg_rate: Rate of regularization
        
        Returns:
            Weight vector via normal equation
        '''
        # self.w = np.linalg.pinv(X) @ y

        self.w = np.zeros((X.shape[1],y.shape[1]))
        eye = np.eye(np.size(X,1))
        self.w = np.linalg.solve(
            reg_rate * eye + X.T @ X,
            X.T @ y,
        )

        return self.w
    
    def calculate_gradient(self, X:np.ndarray, y:np.ndarray, reg_rate:float=0) -> np.ndarray:
        '''Calculate gradients of loss function w.r.t. weight vector on training set.

        Args:
            X: Feature matrix for given inputs
            y: Output label vector as known from the dataset (actual label)
            reg_rate: Rate of regularization
        
        Returns:
            A vector of gradients
        '''
        return np.transpose(X) @ (self.predict(X) - y) + reg_rate * self.w
    
    def update_weights(self, grad:np.ndarray, lr:float) -> np.ndarray:
        '''Update the weights based on the gradient of the loss function.

        Weight updates are carried out with the following formula:
            w_new := w_old - lr * grad
        
        Args:
            grad: Gradient of loss w.r.t w
            lr: Learning rate
        
        Returns:
            Updated weight vector
        '''
        return self.w - lr * grad
    
    # Dynamic learning rate
    def learning_schedule(self, t):
        return self.t0/(t + self.t1)
    
    # GD - Gradient Descent
    def gd(self, X:np.ndarray, y:np.ndarray, num_epochs:int, lr:float, reg_rate:float=0) -> np.ndarray:
        '''Estimates parameters of linar regression model through gradient descent.
        
        Args:
            X: Feature matrix for training data
            y: Output label for training data vector
            num_epochs: Number of training steps
            lr: Learning rate
            reg_rate: Rate of regularization
        
        Returns:
            Weight vector: Final weight vector
        '''
        self.w = np.zeros((X.shape[1],y.shape[1]))
        self.w_all = []
        self.err_all = []
        
        for i in np.arange(0, num_epochs):
            dJdW = self.calculate_gradient(X,y,reg_rate)
            self.w_all.append(self.w)
            self.err_all.append(self.loss(X,y,reg_rate))
            self.w = self.update_weights(dJdW, lr)
        
        return self.w
    
    # MBGD - Mini-Batch Gradient Descent
    def mbgd(self, X:np.ndarray, y:np.ndarray, num_epochs:int, batch_size:int, reg_rate:float=0) -> np.ndarray:
        '''Estimates parameters of linar regression model through mini-batch gradient descent.
        
        Args:
            X: Feature matrix for training data
            y: Output label for training data vector
            num_epochs: No. of epochs (no. of times MBGD is done over the whole training set)
            batch_size: Size of each mini-batch, after which we update the weights
            reg_rate: Rate of regularization
        
        Returns:
            Weight vector: Final weight vector
        '''
        self.w = np.zeros((X.shape[1],y.shape[1]))
        self.w_all = []
        self.err_all = []
        mini_batch_id = 0

        for epoch in range(num_epochs):
            shuffled_indices = np.random.permutation(X.shape[0])
            X_shuffled = X[shuffled_indices]
            y_shuffled = y[shuffled_indices]

            for i in range(0, X.shape[0], batch_size):
                mini_batch_id += 1
                
                # ith mini-batch
                Xi = X_shuffled[i:i + batch_size]
                yi = y_shuffled[i:i + batch_size]

                lr = self.learning_schedule(mini_batch_id)
                gradients = (2/ batch_size) * self.calculate_gradient(Xi, yi, reg_rate)

                self.w_all.append(self.w)
                self.err_all.append(self.loss(Xi, yi, reg_rate))
                self.w = self.update_weights(gradients, lr)
        
        return self.w
    
    # SGD - Stochastic Gradient Descent
    def sgd(self, X:np.ndarray, y:np.ndarray, num_epochs:int, reg_rate:float=0) -> np.ndarray:
        '''Estimates parameters of linar regression model through stochastic gradient descent.
        
        Args:
            X: Feature matrix for training data
            y: Output label for training data vector
            num_epochs: No. of epochs (no. of times MBGD is done over the whole training set)
            reg_rate: Rate of regularization
        
        Returns:
            Weight vector: Final weight vector
        '''
        self.w = np.zeros((X.shape[1],y.shape[1]))
        self.w_all = []
        self.err_all = []

        for epoch in range(num_epochs):
            for i in range(X.shape[0]):
                random_index = np.random.randint(X.shape[0])
                Xi = X[random_index:random_index+1]
                yi = y[random_index:random_index+1]

                lr = self.learning_schedule(epoch * X.shape[0] + i)
                gradients = 2 * self.calculate_gradient(Xi, yi, reg_rate)

                self.w_all.append(self.w)
                self.err_all.append(self.loss(Xi,yi,reg_rate))
                self.w = self.update_weights(gradients, lr)
        
        return self.w


Let's check the estimated weight vector.

1. Normal Equation

In [29]:
lin_reg = LinReg()
W = lin_reg.fit(X_train,Y_train)

# Check if the weight vector is same as the coefficient vector used for making the data:
np.testing.assert_almost_equal(W[1:,:], coef, decimal=2)

In [30]:
print('Predicted Weights: ',W[1:,:]) # The first row only contains the bias term = 1. We are ignoring that.

print('Original Weights: ',coef)

Predicted Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 73.15895218  8.1629982   6.0352084  24.7103234 ]
 [15.95446801 87.17835666 21.92139874 97.58652558 33.68957918]
 [71.40869321 80.17280831 33.94501925 81.48251137  8.01148464]
 [18.21179157 78.96985071 65.87077755 49.81957165 55.53635509]
 [16.74825823 10.45678403 63.64302495 70.64757265  3.15861448]]
Original Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 7

2. Gradient Descent

In [50]:
lin_reg = LinReg()
W = lin_reg.gd(X_train, Y_train, num_epochs=1000, lr=0.01)

# Check if the weight vector is same as the coefficient vector used for making the data:
np.testing.assert_almost_equal(W[1:,:], coef, decimal=2)

In [51]:
print('Predicted Weights: ',W[1:,:]) # The first row only contains the bias term = 1. We are ignoring that.

print('Original Weights: ',coef)

Predicted Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 73.15895218  8.1629982   6.0352084  24.7103234 ]
 [15.95446801 87.17835666 21.92139874 97.58652558 33.68957918]
 [71.40869321 80.17280831 33.94501925 81.48251137  8.01148464]
 [18.21179157 78.96985071 65.87077755 49.81957165 55.53635509]
 [16.74825823 10.45678403 63.64302495 70.64757265  3.15861448]]
Original Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 7

3. Mini Batch Gradient Descent

In [59]:
lin_reg = LinReg()
W = lin_reg.mbgd(X_train, Y_train, num_epochs=1000, batch_size=20)

# Check if the weight vector is same as the coefficient vector used for making the data:
np.testing.assert_almost_equal(W[1:,:], coef, decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

Mismatched elements: 16 / 50 (32%)
Max absolute difference: 0.0346066
Max relative difference: 0.00752033
 x: array([[93.62,  5.16, 54.12, 70.88, 87.08],
       [89.49, 54.79, 81.74, 45.25, 64.38],
       [46.26, 86.8 , 72.71, 74.25, 42.53],...
 y: array([[93.62,  5.2 , 54.13, 70.91, 87.1 ],
       [89.48, 54.76, 81.73, 45.23, 64.36],
       [46.26, 86.83, 72.72, 74.27, 42.55],...

They match till $1$ decimal place, but not after that.

In [60]:
print('Predicted Weights: ',W[1:,:]) # The first row only contains the bias term = 1. We are ignoring that.

print('Original Weights: ',coef)

Predicted Weights:  [[93.61514028  5.16252176 54.12227532 70.88141712 87.07548692]
 [89.48653602 54.78810686 81.73565699 45.25259563 64.37552893]
 [46.25744465 86.79764942 72.71030751 74.24925631 42.53117509]
 [71.92281495 22.87213671 99.63808298 97.49644353 65.04727429]
 [19.94744496 67.9928184   7.21331831  3.04219854 25.74835101]
 [52.64218951 73.16373962  8.16440679  6.04014815 24.71431516]
 [15.95142867 87.15563324 21.91781306 97.57233239 33.67660043]
 [71.40912191 80.17792989 33.9466023  81.48618571  8.01437577]
 [18.20923554 78.94685452 65.86733639 49.80569078 55.52366644]
 [16.74906992 10.45737682 63.64129234 70.64605449  3.15901489]]
Original Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 7

4. Stochastic Gradient Descent

In [57]:
lin_reg = LinReg()
W = lin_reg.sgd(X_train, Y_train, num_epochs=1000)

# Check if the weight vector is same as the coefficient vector used for making the data:
np.testing.assert_almost_equal(W[1:,:], coef, decimal=2)

In [58]:
print('Predicted Weights: ',W[1:,:]) # The first row only contains the bias term = 1. We are ignoring that.

print('Original Weights: ',coef)

Predicted Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 73.15895218  8.1629982   6.0352084  24.7103234 ]
 [15.95446801 87.17835666 21.92139874 97.58652558 33.68957918]
 [71.40869321 80.17280831 33.94501925 81.48251137  8.01148464]
 [18.21179157 78.96985071 65.87077755 49.81957165 55.53635509]
 [16.74825823 10.45678403 63.64302495 70.64757265  3.15861448]]
Original Weights:  [[93.62122462  5.19712837 54.12963353 70.90605195 87.09691237]
 [89.48166561 54.75923762 81.729777   45.23182845 64.35776952]
 [46.26229567 86.82725054 72.71690698 74.27065212 42.54933344]
 [71.92017783 22.84547413 99.63339161 97.47931621 65.03256863]
 [19.95424509 68.02282424  7.2198409   3.06525022 25.76828885]
 [52.64026609 7

**To Do :**
1. Plot graphs of Loss vs Iteration, using some other metric of loss for GD, MBGD, SGD.

2. Solve multi-label regression problem as $k$-independent single label regression problems.

3. Generate dataset with non-linear relationship between input and output. And fit a polynomial regression model with appropriate regularization.